In [20]:
import os
import glob
import fitz
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex


In [21]:
# Ruta al directorio que contiene los archivos PDF
pdf_dir = "C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf"

# Ruta al modelo, por ejemplo, 'clip-ViT-L-14'
model_path = "clip-ViT-L-14"

# Cargar el modelo SentenceTransformer
model = SentenceTransformer(model_path)

In [22]:
def extraer_texto_de_pdf(pdf_path, max_length=77):
    texto = ""
    try:
        with fitz.open(pdf_path) as archivo_pdf:
            for num_pagina in range(len(archivo_pdf)):
                pagina = archivo_pdf[num_pagina]
                texto_pagina = pagina.get_text()
                # Truncar el texto de la página si es necesario
                if len(texto) + len(texto_pagina) > max_length:
                    texto_pagina = texto_pagina[:max_length - len(texto)]
                texto += texto_pagina
    except Exception as e:
        print(f"Error al leer el archivo PDF {pdf_path}: {e}")
    return texto



In [23]:
# Función para buscar en los archivos PDF
def buscar_en_archivos_pdf(consulta):
    archivos = glob.glob(os.path.join(pdf_dir, "*.pdf"))

    # Indexar archivos PDF
    annoy_index = AnnoyIndex(768, metric='angular')
    for idx, archivo in enumerate(archivos):
        texto = extraer_texto_de_pdf(archivo)
        incrustacion = model.encode([texto])[0]
        annoy_index.add_item(idx, incrustacion)
    annoy_index.build(10)

    # Codificar la consulta y encontrar archivos PDF similares
    incrustacion_consulta = model.encode([consulta])[0]
    indices_mas_cercanos, distancias_mas_cercanas = annoy_index.get_nns_by_vector(incrustacion_consulta, 5, include_distances=True)

    # Mostrar o procesar los archivos PDF más cercanos
    resultados = []
    for idx, distancia in zip(indices_mas_cercanos, distancias_mas_cercanas):
        resultados.append((archivos[idx], distancia))
    return resultados




In [24]:
# Consulta de búsqueda
consulta = "ALQUILER DE VEHÍCULOS"

# Realizar la búsqueda
resultados = buscar_en_archivos_pdf(consulta)

# Mostrar resultados
for resultado in resultados:
    print(resultado)

('C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf\\Ejemplo1.pdf', 0.7448084354400635)
('C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf\\Ejemplo2.pdf', 0.7448084354400635)
('C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf\\Ejemplo3.pdf', 0.9151697754859924)
